**Notebook com o scrapping dados para o projeto do site oquecozinhar.com**

# Importação das bibliotecas

In [ ]:
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import numpy as np

# Coleta dos links com as receitas

## Coleta dos links

In [ ]:
# Scrapping do sitemap para coletar os links das receitas
class TudoGostosoLinks():
    def __init__(self, urls, header):
        self.urls = urls
        self.header = header
        self.links = []

    def _get_html(self):
        try:
            self.req = Request(self.url, headers=self.header)
            self.response = urlopen(self.req)
            self.html = self.response.read()
            self.soup = BeautifulSoup(self.html, 'html.parser')
        except:
            pass
        
    def get_links(self):
        for self.url in tqdm(self.urls):
            self._get_html()
            for self.link in self.soup.findAll('loc'):
                self.links.append(self.link.get_text())
        return self.links

In [ ]:
urls = [
    'https://www.tudogostoso.com.br/sitemap-' + str(i) + '.xml'
    for i in range(1, 100)
]
header = {
    'User-Agent':
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'
}

links = TudoGostosoLinks(urls, header)
links = links.get_links()
links

## Preparo dos links

In [ ]:
len(links)

In [ ]:
# Removendo as quebras de parágrafo
links = [link.replace('\n', '') for link in links]

In [ ]:
a = pd.Series(links)
a.to_csv('links.csv')

In [ ]:
# Dividindo os dados, pois o scrapping de tantos sites demora dias
links = pd.read_csv('links.csv', index_col='Unnamed: 0')
links1 = links['0'].iloc[0:7865].tolist()
links2 = links['0'].iloc[7865:15730].tolist()
links3 = links['0'].iloc[15730:23595].tolist()
links4 = links['0'].iloc[23595:31460].tolist()
links5 = links['0'].iloc[31460:39325].tolist()
links6 = links['0'].iloc[39325:47190].tolist()
links7 = links['0'].iloc[47190:55055].tolist()
links8 = links['0'].iloc[55055:62920].tolist()
links9 = links['0'].iloc[62920:70785].tolist()
links10 = links['0'].iloc[70785:78650].tolist()
links11 = links['0'].iloc[78650:86515].tolist()
links12 = links['0'].iloc[86515:94380].tolist()
links13 = links['0'].iloc[94380:102245].tolist()
links14 = links['0'].iloc[102245:110110].tolist()
links15 = links['0'].iloc[110110:117975].tolist()
links16 = links['0'].iloc[117975:125840].tolist()
links17 = links['0'].iloc[125840:133705].tolist()
links18 = links['0'].iloc[133705:141570].tolist()
links19 = links['0'].iloc[141570:149435].tolist()
links20 = links['0'].iloc[149435:157300].tolist()
links21 = links['0'].iloc[157300:165165].tolist()
links22 = links['0'].iloc[165165:173030].tolist()
links23 = links['0'].iloc[173030:180895].tolist()
links24 = links['0'].iloc[180895:188760].tolist()
links25 = links['0'].iloc[188760:196625].tolist()

# Coleta dos dados dos links coletados

## Objeto e preparo

In [ ]:
# Scrapping para coletar os dados dos links
class ColectRecipes():
    def __init__(self, links, header):
        self.links = links
        self.header = header
        self.columns = [
            'LINK', 'RECEITA', 'TIPO', 'PREPARO', 'RENDIMENTO', 'INGREDIENTES',
            'MODO DE PREPARO', 'FAVORITOS'
        ]
        self.dados = pd.DataFrame(data=[[1, 2, 3, 4, 5, 6, 7, 8]],
                                  columns=self.columns)
        self.dados = self.dados.iloc[:0]

#   Coleta o html
    def _get_html(self):
        try:
            self.req = Request(self.url, headers=self.header)
            self.response = urlopen(self.req)
            self.html = self.response.read()
            self.soup = BeautifulSoup(self.html, 'html.parser')
        except:
            pass

#   Coleta os dados e os coloca em um dataframe
    def get_data(self):
        for self.url in tqdm(self.links):
            self._get_html()
            self._get_receita()
            self._get_tipo()
            self._get_preparo()
            self._get_rendimento()
            self._get_ingredientes()
            self._get_modo_preparo()
            self._get_fav()
            self.add = {
                'LINK': self.url,
                'RECEITA': self.receita,
                'TIPO': self.tipo2,
                'PREPARO': self.preparo,
                'RENDIMENTO': self.rendimento,
                'INGREDIENTES': self.ingrediente,
                'MODO DE PREPARO': self.modo_preparo,
                'FAVORITOS': self.fav
            }
            self.dados = self.dados.append(self.add, ignore_index=True)
        return self.dados

    def _get_receita(self):  # Coleta as receitas
        try:
            self.receita = str(self.soup.h1.get_text()).replace('\n', ' ')
        except:
            self.receita = np.nan

    def _get_fav(self):  # Coleta o número de favoritos
        try:
            self.fav = str(
                self.soup.find('data').findNext('data').get_text()).replace(
                    '\n', ' ')
        except:
            self.fav = np.nan

    def _get_tipo(self):  # Coleta o tipo da receita (vegetariano, saudável, etc)
        try:
            self.tipo2 = str(
                self.soup.find('a',
                               class_="type type-box").get_text()).replace(
                                   '\n', ' ')
        except:
            self.tipo2 = np.nan

    def _get_preparo(self):  # Coleta o tempo de preparo
        try:
            self.preparo = str(
                self.soup.find('time',
                               class_="dt-duration").get_text()).replace(
                                   '\n', ' ')
        except:
            self.preparo = np.nan

    def _get_rendimento(self):  # Coleta o número de porções
        try:
            self.rendimento = str(
                self.soup.find('data',
                               class_="p-yield num yield").get_text()).replace(
                                   '\n', ' ')
        except:
            self.rendimento = np.nan

    def _get_ingredientes(self):  # Coleta os ingredientes
        try:
            self.ingrediente = {}
            self.div_ingredientes = self.soup.find(
                'div', class_="col-lg-8 ingredients-card")
            
#           Como existem variações da forma que os dados são inseridos no site,
#           foi necessário prever algumas situações possíveis para os dados não ficarem confusos
            if len(self.div_ingredientes.findAll('h3')) == 0:  # Caso não existam subdivisões
                self.ingrediente_aux = [
                    str(self.ing.get_text()).replace('\n', ' ') for self.ing in
                    self.div_ingredientes.findNext('ul').findAll('li')
                ]
                self.ingrediente = self.ingrediente_aux.copy()
            else:  # Caso existam subdivisões
                for self.item in self.div_ingredientes.findAll('h3'):
                    self.tipo = str(self.item.get_text()).replace('\n', ' ')
                    self.ingrediente_aux = [
                        str(self.ing.get_text()).replace('\n', ' ')
                        for self.ing in self.item.findNext('ul').findAll('li')
                    ]
                    self.ingrediente[self.tipo] = self.ingrediente_aux
        except:
            self.ingrediente = np.nan

    def _get_modo_preparo(self):  # Coleta o modo de preparo
        try:
            self.modo_preparo = {}
            self.div_modo_preparo = self.soup.find(
                'div', class_="instructions e-instructions")
            
#           Como existem variações da forma que os dados são inseridos no site,
#           foi necessário prever algumas situações possíveis para os dados não ficarem confusos
            if len(self.div_modo_preparo.findAll('h3')) == 0:  # Caso não existam subdivisões
                self.modo_preparo_aux = [
                    str(self.mp.get_text()).replace('\n', ' ') for self.mp in
                    self.div_modo_preparo.findNext('ol').findAll('li')
                ]
                self.modo_preparo = self.modo_preparo_aux.copy()
            elif len(self.div_modo_preparo.findAll('h3')) == 1:  # Caso exista 1 subdivisão
                self.modo_preparo['Receita'] = [
                    str(self.mp.get_text()).replace('\n', ' ') for self.mp in
                    self.div_modo_preparo.findNext('ol').findAll('li')
                ]
                self.modo_preparo[str(
                    self.div_modo_preparo.findNext('h3').get_text()).replace(
                        '\n', ' ')] = [
                            str(self.mp.get_text()).replace('\n', ' ')
                            for self.mp in self.div_modo_preparo.findNext(
                                'h3').findNext('li').findAll('span')
                        ]
            else:  # Caso existam várias subdivisões
                for self.item in self.div_modo_preparo.findAll('h3'):
                    self.tipo = str(self.item.get_text()).replace('\n', ' ')
                    if self.tipo == 'Informações Adicionais':
                        self.modo_preparo[self.tipo] = [
                            str(self.mp.get_text()).replace('\n', ' ') for
                            self.mp in self.item.findNext('li').findAll('span')
                        ]
                    else:
                        self.modo_preparo_aux = [
                            str(self.mp.get_text()).replace('\n', ' ') for
                            self.mp in self.item.findNext('ol').findAll('li')
                        ]
                        self.modo_preparo[self.tipo] = self.modo_preparo_aux
        except:
            self.modo_preparo = np.nan

    def show_data(self):  # Exibe os dados
        return self.dados

In [ ]:
header = {
    'User-Agent':
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'
}

Os dados foram divididos em 25 partes para evitar erros e dividir o trabalho em curtos espaços de tempo.

Cada parte demorou de 3 a 4 horas pra ser executada

## Scrapping

In [ ]:
dados = ColectRecipes(links1, header)
df = dados.get_data()
df.to_json('ReceitasTudoGostoso1.json', orient='records')

In [ ]:
dados = ColectRecipes(links2, header)
df = dados.get_data()
df.to_json('ReceitasTudoGostoso2.json', orient='records')

In [ ]:
dados = ColectRecipes(links3, header)
df = dados.get_data()
df.to_json('ReceitasTudoGostoso3.json', orient='records')

In [ ]:
dados = ColectRecipes(links4, header)
df = dados.get_data()
df.to_json('ReceitasTudoGostoso4.json', orient='records')

In [ ]:
dados = ColectRecipes(links5, header)
df = dados.get_data()
df.to_json('ReceitasTudoGostoso5.json', orient='records')

In [ ]:
dados = ColectRecipes(links6, header)
df = dados.get_data()
df.to_json('ReceitasTudoGostoso6.json', orient='records')

In [ ]:
dados = ColectRecipes(links7, header)
df = dados.get_data()
df.to_json('ReceitasTudoGostoso7.json', orient='records')

In [ ]:
dados = ColectRecipes(links8, header)
df = dados.get_data()
df.to_json('ReceitasTudoGostoso8.json', orient='records')

In [ ]:
dados = ColectRecipes(links9, header)
df = dados.get_data()
df.to_json('ReceitasTudoGostoso9.json', orient='records')

In [ ]:
dados = ColectRecipes(links10, header)
df = dados.get_data()
df.to_json('ReceitasTudoGostoso10.json', orient='records')

In [ ]:
dados = ColectRecipes(links11, header)
df = dados.get_data()
df.to_json('ReceitasTudoGostoso11.json', orient='records')

In [ ]:
dados = ColectRecipes(links12, header)
df = dados.get_data()
df.to_json('ReceitasTudoGostoso12.json', orient='records')

In [ ]:
dados = ColectRecipes(links13, header)
df = dados.get_data()
df.to_json('ReceitasTudoGostoso13.json', orient='records')

In [ ]:
dados = ColectRecipes(links14, header)
df = dados.get_data()
df.to_json('ReceitasTudoGostoso14.json', orient='records')

In [ ]:
dados = ColectRecipes(links15, header)
df = dados.get_data()
df.to_json('ReceitasTudoGostoso15.json', orient='records')

In [ ]:
dados = ColectRecipes(links16, header)
df = dados.get_data()
df.to_json('ReceitasTudoGostoso16.json', orient='records')

In [ ]:
dados = ColectRecipes(links17, header)
df = dados.get_data()
df.to_json('ReceitasTudoGostoso17.json', orient='records')

In [ ]:
dados = ColectRecipes(links18, header)
df = dados.get_data()
df.to_json('ReceitasTudoGostoso18.json', orient='records')

In [ ]:
dados = ColectRecipes(links19, header)
df = dados.get_data()
df.to_json('ReceitasTudoGostoso19.json', orient='records')

In [ ]:
dados = ColectRecipes(links20, header)
df = dados.get_data()
df.to_json('ReceitasTudoGostoso20.json', orient='records')

In [ ]:
dados = ColectRecipes(links21, header)
df = dados.get_data()
df.to_json('ReceitasTudoGostoso21.json', orient='records')

In [ ]:
dados = ColectRecipes(links22, header)
df = dados.get_data()
df.to_json('ReceitasTudoGostoso22.json', orient='records')

In [ ]:
dados = ColectRecipes(links23, header)
df = dados.get_data()
df.to_json('ReceitasTudoGostoso23.json', orient='records')

In [ ]:
dados = ColectRecipes(links24, header)
df = dados.get_data()
df.to_json('ReceitasTudoGostoso24.json', orient='records')

In [ ]:
dados = ColectRecipes(links25, header)
df = dados.get_data()
df.to_json('ReceitasTudoGostoso25.json', orient='records')